In [ ]:
# If running from Colab, please uncomment and run the following cell to get summer installed.
# !pip install summerepi2==1.3.5 estival==0.5.1

In [ ]:
# This is required for pymc parallel evaluation in notebooks
import multiprocessing as mp
import platform

if platform.system() != "Windows":
    mp.set_start_method('forkserver')

In [ ]:
# Basic imports for summer2 compartmental modelling
import numpy as np
import pandas as pd
from datetime import datetime
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
pd.options.plotting.backend = 'plotly'

In [ ]:
# Estival is our calibration/optimization framework - for connecting models and parameters to data
# The following imports are the 'building blocks' of estival models

# Targets represent data we are trying to fit to
from estival import targets as est

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

In [ ]:
# PyMC imports - PyMC is a widely used probabilistic programming framework for Python
from estival.wrappers import pymc as epm
import pymc as pm

## Model construction and execution
The following cells constitute the epidemiological analysis of this notebook.
The code is intended to be very simple and adaptable.
Please feel free to adjust any of the data sources or parameters,
which is the purpose of this notebook.
### Country selection
First choose your country using it's ISO code,
with options only available for Malaysia, the Philippines and Vietnam.
This will determine the Our World in Data (OWID) case data
that can be used to compare your model results against.
You can use our approximate estimate of the total population of the country,
or replace the value as per your preference.
### Dates
Feel free to edit the datetime objects that specify
the start and end dates for the analysis, the reference ("zero") date
and the left limit for the x-axis in the final plot.
### Model
The model is a parameterised summer2 SEIR model
with frequency-dependent transmission and
partial observation of incidence (termed notifications).
### Results
The SEIR model provides a poor-to-moderate fit to the target data
for the Omicron waves in each of the three countries.
Please adjust parameters and dates to achieve a better fit for your country.

In [ ]:
iso = 'MYS'

In [ ]:
cases_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv', index_col=0)[iso]
cases_data.index = pd.to_datetime(cases_data.index)
approx_pops = {
    'MYS': 33e6,
    'PHL': 114e6,
    'VNM': 97e6,
}

In [ ]:
analysis_start_date = datetime(2022, 1, 1)
analysis_end_date = datetime(2022, 6, 1)
epi_model = CompartmentalModel(
    [analysis_start_date, analysis_end_date],
    ['susceptible', 'exposed', 'infectious', 'recovered'],
    ['infectious'],
    ref_date=datetime(2019, 12, 31),
)
epi_model.add_infection_frequency_flow('infection', Parameter('contact_rate'), 'susceptible', 'exposed')
epi_model.add_transition_flow('progression', 1.0 / Parameter('incubation_period'), 'exposed', 'infectious')
epi_model.add_transition_flow('recovery', 1.0 / Parameter('infectious_period'), 'infectious', 'recovered')
epi_model.set_initial_population({'susceptible': approx_pops[iso], 'infectious': Parameter('initial_infected')})
incidence = epi_model.request_output_for_flow('incidence', 'progression', save_results=False)
epi_model.request_function_output('notifications', incidence * Parameter('detection_prop'));

In [ ]:
parameters = {
    'contact_rate': 0.4,
    'incubation_period': 5.0,
    'infectious_period': 5.0,
    'detection_prop': 0.07,
    'initial_infected': 100000.0,
}
epi_model.run(parameters)

In [ ]:
plot_start_date = datetime(2021, 10, 1)
comparison_df = pd.concat([epi_model.get_derived_outputs_df(), cases_data])
comparison_df.columns = ['modelled', 'reported']
comparison_plot = comparison_df.plot()
comparison_plot.update_xaxes(range=(plot_start_date, analysis_end_date))

In [ ]:
# Select a subset of the data which will be used for model evaluation during calibration
cases_data_select = cases_data["jan 2022":"may 2022"]
cases_data_select.plot()

In [ ]:
# Describe a 'target'; some observed data against which the model will be evaluated
# (and a description of how this evaluation will be performed)
targets = [
    est.TruncatedNormalTarget("notifications", cases_data_select, (0.0,np.inf),
        esp.UniformPrior("notifications_dispersion",(0.1, cases_data_select.max()*0.1)))
]

In [ ]:
# Describe priors - the ranges (and statistical distributions) of what we believe
# the parameters might (or should) be
priors = [
    esp.UniformPrior("contact_rate", (0.01,1.0)),
    esp.TruncNormalPrior("incubation_period", 5.0,1.0, (1.0,10.0)),
    esp.TruncNormalPrior("infectious_period", 5.0,1.0, (1.0,10.0)),
    esp.UniformPrior("detection_prop", (0.01,0.5)),
]

In [ ]:
# The BayesianCompartmentalModel class is the primary entry point to all optimization and calibration
# methods in estival
# It takes a CompartmentalModel object, default parameters, priors, and targets
# The default parameters will be used as fixed values when no prior is specified for a given parameter

bcm = BayesianCompartmentalModel(epi_model, parameters, priors, targets)

In [ ]:
with pm.Model() as model:

    # This is all you need - a single call to use_model
    variables = epm.use_model(bcm)

    # Now call a sampler using the variables from use_model
    # In this case we use the Differential Evolution Metropolis(Z) sampler
    # See the PyMC docs for more details
    idata = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=4000, tune=2000,cores=4,chains=4)

In [ ]:
# Arviz is a library for examining MCMC outputs and producing plots/summary statistics/wrangling data
# It supports a variety of frameworks (including PyMC, which we use above for producing our samples)

import arviz as az

In [ ]:
# Get summary stats for the calibration
# These are useful to give a rough assessment of the quality of the outputs
az.summary(idata)

In [ ]:
# Plot the traces (the values of the parameters at each iteration)
az.plot_trace(idata, figsize=(16,3.2*len(idata.posterior)),compact=False);

In [ ]:
# Use the arviz extract method to obtain some samples
sample_idata = az.extract(idata, num_samples = 100)

In [ ]:
# Estival includes a variety of tools for working with parameter sets
from estival.sampling import tools as esamptools

In [ ]:
# Get the modelled results for our sampled parameters
mres = esamptools.model_results_for_samples(sample_idata, bcm)

In [ ]:
# Plot the uncertainty quantiles of the above sampled model results, against case data
pd.options.plotting.backend="matplotlib"
esamptools.quantiles_for_results(mres.results,[0.01,0.25,0.5,0.75,0.99])["notifications"].plot()
cases_data_select.plot(style='.',color='black')